In [1]:
# --- 0. Setup ---
!pip install diffusers accelerate transformers safetensors torch torchvision scikit-learn


In [2]:
!pip install --upgrade diffusers transformers torch
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 127.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 20.3 MB/s eta 0:00:00


In [4]:
#torch.cuda.empty_cache()
!pip install accelerate

In [24]:
# --- 0. Install dependencies ---
import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

# --- 1. Device and RNG ---
device = "cuda"
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)
torch.cuda.empty_cache()

# --- 2. Load SSD-1B pipeline ---
pipe = StableDiffusionPipeline.from_pretrained(
    "segmind/SSD-1B",
    safety_checker=None  # optional; research only
).to(device)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 2b. Cast UNet to float16 to avoid dtype mismatch ---
pipe.unet = pipe.unet.half().to(device)

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"

prompt_embeds, _ = pipe.encode_prompt(
    prompt=prompt,
    device=device,
    num_images_per_prompt=1,
    do_classifier_free_guidance=False
)
prompt_embeds = prompt_embeds.half()  # ensure float16
print(f"prompt_embeds shape: {prompt_embeds.shape}")

# --- 3b. Correct time_ids dimension for UNet ---
batch_size, seq_len, hidden_dim = prompt_embeds.shape
# Create time_ids with the same 3 dimensions as prompt_embeds
time_ids = torch.zeros((batch_size, seq_len, hidden_dim), device=device, dtype=prompt_embeds.dtype)

added_cond_kwargs = {
    "text_embeds": prompt_embeds,  # [B, seq_len, hidden_dim]
    "time_ids": time_ids            # [B, seq_len, hidden_dim], matching dimensions
}

# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),  # reduced resolution
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds,
        added_cond_kwargs=added_cond_kwargs
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 7. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 8. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 9. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12, 4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1, 6, i + 1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 10. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")

Using device: cuda


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


prompt_embeds shape: torch.Size([1, 77, 768])


RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [21]:
# --- 0. Install dependencies ---


import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

# --- 1. Device and RNG ---
device = "cuda"
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)
torch.cuda.empty_cache()

# --- 2. Load SSD-1B pipeline ---
pipe = StableDiffusionPipeline.from_pretrained(
    "segmind/SSD-1B",
    safety_checker=None  # optional; research only
).to(device)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 2b. Cast UNet to float16 to avoid dtype mismatch ---
pipe.unet = pipe.unet.half().to(device)

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"

prompt_embeds, _ = pipe.encode_prompt(
    prompt=prompt,
    device=device,
    num_images_per_prompt=1,
    do_classifier_free_guidance=False
)
prompt_embeds = prompt_embeds.half()  # ensure float16
print(f"prompt_embeds shape: {prompt_embeds.shape}")

# --- 3b. Correct 2D time_ids for UNet ---
batch_size, seq_len, hidden_dim = prompt_embeds.shape
time_ids = torch.zeros((batch_size, hidden_dim), device=device, dtype=prompt_embeds.dtype)  # [B, H]

added_cond_kwargs = {
    "text_embeds": prompt_embeds,  # [B, seq_len, H]
    "time_ids": time_ids            # [B, H], leave 2D
}

# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),  # reduced resolution
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds,
        added_cond_kwargs=added_cond_kwargs
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 7. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 8. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 9. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12, 4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1, 6, i + 1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 10. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")


Using device: cuda


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


prompt_embeds shape: torch.Size([1, 77, 768])


RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [20]:
# --- 0. Install dependencies ---
import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

# --- 1. Device and RNG ---
device = "cuda"
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)
torch.cuda.empty_cache()

# --- 2. Load SSD-1B pipeline ---
pipe = StableDiffusionPipeline.from_pretrained(
    "segmind/SSD-1B",
    safety_checker=None  # optional; research only
).to(device)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 2b. Cast UNet to float16 to avoid dtype mismatch ---
pipe.unet = pipe.unet.half().to(device)

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"

prompt_embeds, _ = pipe.encode_prompt(
    prompt=prompt,
    device=device,
    num_images_per_prompt=1,
    do_classifier_free_guidance=False
)
prompt_embeds = prompt_embeds.half()  # ensure float16
print(f"prompt_embeds shape: {prompt_embeds.shape}")

# --- 3b. Minimal 3D-compatible added_cond_kwargs for SSD-1B UNet ---
batch_size, seq_len, hidden_dim = prompt_embeds.shape
# Create time_ids with 3 dimensions and expand to match seq_len
time_ids = torch.zeros((batch_size, 1, hidden_dim), device=device, dtype=torch.float16)  # [B, 1, H]
time_ids = time_ids.expand(-1, seq_len, -1)  # [B, seq_len, H]

added_cond_kwargs = {
    "text_embeds": prompt_embeds,  # [B, seq_len, H]
    "time_ids": time_ids           # [B, seq_len, H], compatible with text_embeds
}

# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),  # reduced resolution
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds,
        added_cond_kwargs=added_cond_kwargs
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 7. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 8. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 9. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12, 4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1, 6, i + 1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 10. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")

Using device: cuda


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


prompt_embeds shape: torch.Size([1, 77, 768])


RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [17]:
# --- 0. Install dependencies ---
!pip install diffusers accelerate transformers safetensors torch torchvision scikit-learn imageio -q

import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

# --- 1. Device and RNG ---
device = "cuda"
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)
torch.cuda.empty_cache()

# --- 2. Load SSD-1B pipeline ---
pipe = StableDiffusionPipeline.from_pretrained(
    "segmind/SSD-1B",
    safety_checker=None  # optional; research only
).to(device)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 2b. Cast UNet to float16 to avoid dtype mismatch ---
pipe.unet = pipe.unet.half().to(device)

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"

prompt_embeds, _ = pipe.encode_prompt(
    prompt=prompt,
    device=device,
    num_images_per_prompt=1,
    do_classifier_free_guidance=False
)
prompt_embeds = prompt_embeds.half()  # ensure float16
print(f"prompt_embeds shape: {prompt_embeds.shape}")

# --- 3b. Minimal added_cond_kwargs for SSD-1B UNet ---
added_cond_kwargs = {
    "text_embeds": prompt_embeds,  # required for text_time
    "time_ids": torch.zeros((1, 6), device=device, dtype=torch.float16)  # placeholder
}

# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),  # reduced resolution
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds,
        added_cond_kwargs=added_cond_kwargs  # include minimal dict
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 7. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 8. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 9. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12, 4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1, 6, i + 1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 10. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")


Using device: cuda


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


prompt_embeds shape: torch.Size([1, 77, 768])


RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [16]:
# --- 0. Install dependencies ---
import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

# --- 1. Device and RNG ---
device = "cuda"
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)
torch.cuda.empty_cache()

# --- 2. Load SSD-1B pipeline ---
pipe = StableDiffusionPipeline.from_pretrained(
    "segmind/SSD-1B",
    safety_checker=None  # optional; research only
).to(device)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 2b. Cast UNet to float16 to avoid dtype mismatch ---
pipe.unet = pipe.unet.half().to(device)

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"

prompt_embeds, _ = pipe.encode_prompt(
    prompt=prompt,
    device=device,
    num_images_per_prompt=1,
    do_classifier_free_guidance=False
)
prompt_embeds = prompt_embeds.half()  # ensure float16
print(f"prompt_embeds shape: {prompt_embeds.shape}")

# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),  # reduced resolution
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds,
        added_cond_kwargs={} # Explicitly pass an empty dictionary
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 7. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 8. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 9. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12, 4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1, 6, i + 1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 10. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")

Using device: cuda


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


prompt_embeds shape: torch.Size([1, 77, 768])


ValueError: <class 'diffusers.models.unets.unet_2d_condition.UNet2DConditionModel'> has the config param `addition_embed_type` set to 'text_time' which requires the keyword argument `text_embeds` to be passed in `added_cond_kwargs`

In [11]:
# --- 0. Install dependencies ---
!pip install diffusers accelerate transformers safetensors torch torchvision scikit-learn imageio -q

import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

# --- 1. Device and RNG ---
device = "cuda"
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)
torch.cuda.empty_cache()

# --- 2. Load SSD-1B pipeline ---
pipe = StableDiffusionPipeline.from_pretrained(
    "segmind/SSD-1B",
    safety_checker=None  # optional; research only
).to(device)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 2b. Cast UNet to float16 to avoid dtype mismatch ---
pipe.unet = pipe.unet.half().to(device)

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"

prompt_embeds, _ = pipe.encode_prompt(
    prompt=prompt,
    device=device,
    num_images_per_prompt=1,
    do_classifier_free_guidance=False
)
prompt_embeds = prompt_embeds.half()  # ensure float16
print(f"prompt_embeds shape: {prompt_embeds.shape}")

# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),  # reduced resolution
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 7. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 8. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 9. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12, 4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1, 6, i + 1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 10. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")


Using device: cuda


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


prompt_embeds shape: torch.Size([1, 77, 768])


TypeError: argument of type 'NoneType' is not iterable

In [10]:
# --- 0. Install dependencies ---
!pip install diffusers accelerate transformers safetensors torch torchvision scikit-learn imageio -q

import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

# --- 1. Device and RNG ---
device = "cuda"
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)

torch.cuda.empty_cache()

# --- 2. Load SSD-1B pipeline ---
pipe = StableDiffusionPipeline.from_pretrained(
    "segmind/SSD-1B",
    safety_checker=None  # optional; for research only
).to(device)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"

# encode_prompt returns a tuple; unpack and convert to float16
prompt_embeds, _ = pipe.encode_prompt(
    prompt=prompt,
    device=device,
    num_images_per_prompt=1,
    do_classifier_free_guidance=False
)
prompt_embeds = prompt_embeds.half()  # ensure float16
print(f"prompt_embeds shape: {prompt_embeds.shape}")

# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),  # reduced resolution
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds  # already float16
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 7. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 8. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 9. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12, 4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1, 6, i + 1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 10. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")


Using device: cuda


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


prompt_embeds shape: torch.Size([1, 77, 768])


RuntimeError: mat1 and mat2 must have the same dtype, but got Half and Float

In [9]:
# --- 0. Install dependencies ---
import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

# --- 1. Device and RNG ---
device = "cuda"
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)

torch.cuda.empty_cache()

# --- 2. Load SSD-1B pipeline ---
pipe = StableDiffusionPipeline.from_pretrained(
    "segmind/SSD-1B",
    torch_dtype=torch.float16,       # updated API
).to(device)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"

prompt_embeds, _ = pipe.encode_prompt(
    prompt=prompt,
    device=device,
    num_images_per_prompt=1,
    do_classifier_free_guidance=False
)

# Explicitly cast prompt_embeds to float16
prompt_embeds = prompt_embeds.to(torch.float16)

print(f"prompt_embeds shape: {prompt_embeds.shape}")

# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),  # reduced resolution
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 7. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 8. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 9. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12, 4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1, 6, i + 1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 10. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")

Using device: cuda


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


prompt_embeds shape: torch.Size([1, 77, 768])


TypeError: argument of type 'NoneType' is not iterable

In [5]:
# --- 0. Setup ---
#!pip install diffusers accelerate transformers safetensors torch torchvision scikit-learn imageio -q

import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

# --- 1. Device and RNG ---
device = "cuda"
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)

torch.cuda.empty_cache()

# --- 2. Load SSD-1B pipeline ---
pipe = StableDiffusionPipeline.from_pretrained(
    "segmind/SSD-1B",
    torch_dtype=torch.float16
).to(device)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"
prompt_embeds = pipe.encode_prompt(prompt)
print(f"prompt_embeds shape: {prompt_embeds.shape}")

# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 7. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 8. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 9. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12, 4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1, 6, i + 1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 10. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")


Using device: cuda


model_index.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/5.33G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


TypeError: StableDiffusionPipeline.encode_prompt() missing 3 required positional arguments: 'device', 'num_images_per_prompt', and 'do_classifier_free_guidance'

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionXLPipeline
from sklearn.decomposition import TruncatedSVD
import imageio
import os

# --- 1. Device and RNG ---
device = "cuda"
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)

# Clear GPU memory
torch.cuda.empty_cache()

# --- 2. Load SDXL pipeline ---
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
).to(device, dtype=torch.float16)

# Enable memory optimizations
pipe.enable_model_cpu_offload()  # Primary memory fix
pipe.enable_vae_tiling()
pipe.enable_xformers_memory_efficient_attention()  # Uncomment if xformers is installed
vae = pipe.vae
print("U-Net config:", pipe.unet.config)

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"
prompt_embeds, _, pooled_embeds, _ = pipe.encode_prompt(
    prompt=prompt,
    device=device,
    num_images_per_prompt=1,
    do_classifier_free_guidance=False
)
print(f"prompt_embeds shape: {prompt_embeds.shape}")
print(f"pooled_embeds shape: {pooled_embeds.shape}")
time_ids = torch.tensor([[1024, 1024, 0, 0, 1024, 1024]], device=device, dtype=torch.float16)
print(f"time_ids shape: {time_ids.shape}")

# Concatenate pooled_embeds and time_ids for added_cond_kwargs
added_cond_embeds = torch.cat([pooled_embeds, time_ids], dim=-1)
print(f"added_cond_embeds shape: {added_cond_embeds.shape}")


# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64), # Reduced resolution
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    print(f"Step {i}, latent_model_input shape: {latent_model_input.shape}")
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds,
        added_cond_kwargs={
            "text_embeds": added_cond_embeds, # Use the concatenated embeddings
            "time_ids": time_ids # Still need time_ids for other parts of the UNet
        }
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 7. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 8. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 9. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12, 4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1, 6, i + 1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 10. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")

In [26]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionXLPipeline
from sklearn.decomposition import TruncatedSVD
import imageio
import os

# Set PyTorch memory configuration - already set in a previous cell, keep for clarity if this is run independently
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# --- 1. Device and RNG ---
device = "cuda"
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)

# Clear GPU memory
torch.cuda.empty_cache()

# --- 2. Load SDXL pipeline ---
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
).to(device, dtype=torch.float16)

# Enable memory optimizations
pipe.enable_model_cpu_offload()  # Primary memory fix
pipe.enable_vae_tiling()
pipe.enable_xformers_memory_efficient_attention() # Explicitly enable xformers attention
vae = pipe.vae
print("U-Net config:", pipe.unet.config)

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"
# SDXL pipeline's encode_prompt handles both text encoders and returns all necessary embeddings
prompt_embeds, negative_prompt_embeds, pooled_prompt_embeds, negative_pooled_prompt_embeds = pipe.encode_prompt(
    prompt=prompt,
    device=device,
    num_images_per_prompt=1,
    do_classifier_free_guidance=False # Assuming no classifier-free guidance for simplicity as in original code
)

print(f"prompt_embeds shape: {prompt_embeds.shape}")
print(f"pooled_prompt_embeds shape: {pooled_prompt_embeds.shape}")

# Time IDs for 1024x1024 resolution - standard for SDXL base
# Structure is [original_height, original_width, crops_coords_top, crops_coords_left, target_height, target_width]
time_ids = torch.tensor([[1024, 1024, 0, 0, 1024, 1024]], device=device, dtype=torch.float16)
print(f"time_ids shape: {time_ids.shape}")

# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64), # Reduced resolution (512x512 equivalent)
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    # print(f"Step {i}, latent_model_input shape: {latent_model_input.shape}") # Optional debug print

    # added_cond_kwargs for SDXL
    added_cond_kwargs = {
        "text_embeds": pooled_prompt_embeds, # Pooled embeddings
        "time_ids": time_ids # Time IDs
    }

    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds, # Per-token embeddings
        added_cond_kwargs=added_cond_kwargs
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    # Ensure probes are also float16 for VAE decoding
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- Handle NaNs before SVD ---
if np.isnan(X).any():
    print(f"Found {np.isnan(X).sum()} NaN values in X. Removing rows with NaNs.")
    X = X[~np.isnan(X).any(axis=1)]
    if X.shape[0] == 0:
        print("All rows contained NaNs. Cannot perform SVD.")
    else:
        # --- 7. SVD resonance analysis ---
        svd = TruncatedSVD(n_components=min(10, X.shape[0])) # Ensure n_components is not more than samples
        svd.fit(X)
        singular_vectors = svd.components_
        print("Explained variance ratios:", svd.explained_variance_ratio_)

        # --- 8. Animate along singular directions ---
        def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
            z = z_start.clone().detach()
            # Ensure latent_dir is also float16
            latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
            latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat) # np.linalg.norm works on float, result is scalar

            frames = []
            for i in range(n_frames):
                z = z + step_size * latent_dir
                with torch.no_grad():
                    # Ensure z is float16 for VAE decoding
                    out = vae.decode((z / vae.config.scaling_factor).half()).sample
                img = (out.clamp(-1, 1) + 1) / 2
                # Ensure output is converted to uint8 correctly
                frames.append((img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))
            return frames

        big_mode = singular_vectors[0]
        small_mode = singular_vectors[-1]

        frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
        frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

        # --- 9. Display first frames ---
        def show_frames(frames, title):
            plt.figure(figsize=(12, 4))
            for i, f in enumerate(frames[:6]):
                plt.subplot(1, 6, i + 1)
                plt.imshow(f)
                plt.axis("off")
            plt.suptitle(title, fontsize=14)
            plt.show()

        show_frames(frames_big, "BIG singular mode")
        show_frames(frames_small, "SMALL singular mode")

        # --- 10. Export GIFs ---
        # Ensure data is correctly formatted for imageio
        imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
        imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

        print("GIFs saved: animation_big.gif, animation_small.gif")

else:
    print("No NaN values found in X. Proceeding with SVD.")
    # --- 7. SVD resonance analysis ---
    svd = TruncatedSVD(n_components=min(10, X.shape[0])) # Ensure n_components is not more than samples
    svd.fit(X)
    singular_vectors = svd.components_
    print("Explained variance ratios:", svd.explained_variance_ratio_)

    # --- 8. Animate along singular directions ---
    def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
        z = z_start.clone().detach()
        # Ensure latent_dir is also float16
        latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
        latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat) # np.linalg.norm works on float, result is scalar

        frames = []
        for i in range(n_frames):
            z = z + step_size * latent_dir
            with torch.no_grad():
                # Ensure z is float16 for VAE decoding
                out = vae.decode((z / vae.config.scaling_factor).half()).sample
            img = (out.clamp(-1, 1) + 1) / 2
            # Ensure output is converted to uint8 correctly
            frames.append((img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))
        return frames


    big_mode = singular_vectors[0]
    small_mode = singular_vectors[-1]

    frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
    frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

    # --- 9. Display first frames ---
    def show_frames(frames, title):
        plt.figure(figsize=(12, 4))
        for i, f in enumerate(frames[:6]):
            plt.subplot(1, 6, i + 1)
            plt.imshow(f)
            plt.axis("off")
        plt.suptitle(title, fontsize=14)
        plt.show()

    show_frames(frames_big, "BIG singular mode")
    show_frames(frames_small, "SMALL singular mode")

    # --- 10. Export GIFs ---
    # Ensure data is correctly formatted for imageio
    imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
    imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

    print("GIFs saved: animation_big.gif, animation_small.gif")

Using device: cuda


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

U-Net config: FrozenDict({'sample_size': 128, 'in_channels': 4, 'out_channels': 4, 'center_input_sample': False, 'flip_sin_to_cos': True, 'freq_shift': 0, 'down_block_types': ['DownBlock2D', 'CrossAttnDownBlock2D', 'CrossAttnDownBlock2D'], 'mid_block_type': 'UNetMidBlock2DCrossAttn', 'up_block_types': ['CrossAttnUpBlock2D', 'CrossAttnUpBlock2D', 'UpBlock2D'], 'only_cross_attention': False, 'block_out_channels': [320, 640, 1280], 'layers_per_block': 2, 'downsample_padding': 1, 'mid_block_scale_factor': 1, 'dropout': 0.0, 'act_fn': 'silu', 'norm_num_groups': 32, 'norm_eps': 1e-05, 'cross_attention_dim': 2048, 'transformer_layers_per_block': [1, 2, 10], 'reverse_transformer_layers_per_block': None, 'encoder_hid_dim': None, 'encoder_hid_dim_type': None, 'attention_head_dim': [5, 10, 20], 'num_attention_heads': None, 'dual_cross_attention': False, 'use_linear_projection': True, 'class_embed_type': None, 'addition_embed_type': 'text_time', 'addition_time_embed_dim': 256, 'num_class_embeds': 

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 79.32 GiB of which 1.88 MiB is free. Process 14864 has 79.31 GiB memory in use. Of the allocated memory 77.29 GiB is allocated by PyTorch, and 1.52 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionXLPipeline
from sklearn.decomposition import TruncatedSVD
import imageio
torch.cuda.empty_cache()
# --- 1. Device and RNG ---
device = "cuda"
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)

# --- 2. Load SDXL pipeline ---
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
).to(device, dtype=torch.float16)

pipe.enable_vae_tiling()
vae = pipe.vae
print("U-Net config:", pipe.unet.config)

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"

# Use pipe.encode_prompt to handle both text encoders
prompt_embeds, _, pooled_embeds, _ = pipe.encode_prompt(
    prompt=prompt,
    device=device,
    num_images_per_prompt=1,
    do_classifier_free_guidance=False
)

# Ensure shapes are correct
print(f"prompt_embeds shape: {prompt_embeds.shape}")  # Expected: (1, 77, 2048)
print(f"pooled_embeds shape: {pooled_embeds.shape}")  # Expected: (1, 1280)
# Time IDs for 1024x1024
time_ids = torch.tensor([[1024, 1024, 0, 0, 1024, 1024]], device=device, dtype=torch.float16)
print(f"time_ids shape: {time_ids.shape}")

# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 128, 128),
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    print(f"Step {i}, latent_model_input shape: {latent_model_input.shape}")
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds,
        added_cond_kwargs={
            "text_embeds": pooled_embeds,
            "time_ids": time_ids
        }
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 7. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 8. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 9. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12, 4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1, 6, i + 1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 10. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionXLPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

# --- 1. Device and RNG ---
device = "cuda"
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)

# --- 2. Load SDXL pipeline ---
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    # variant="fp16"  # Try without variant to rule out issues
).to(device, dtype=torch.float16)

pipe.enable_vae_tiling()
vae = pipe.vae
print("U-Net config:", pipe.unet.config)  # Debug U-Net configuration

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"
prompt_embeds, _, _, _ = pipe.encode_prompt(prompt)
text_inputs = pipe.tokenizer(prompt, return_tensors="pt").input_ids.to(device)
pooled_embeds = pipe.text_encoder(
    text_inputs,
    output_hidden_states=True
).pooler_output.to(torch.float16)

# Debug shapes
print(f"prompt_embeds[0] shape: {prompt_embeds[0].shape}")
print(f"pooled_embeds shape: {pooled_embeds.shape}")

# Time IDs for 1024x1024
time_ids = torch.tensor([[1024, 1024, 0, 0, 1024, 1024]], device=device, dtype=torch.float16)
print(f"time_ids shape: {time_ids.shape}")

# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 128, 128),  # 1024x1024
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

# Add hook to debug U-Net layers
def hook_fn(module, input, output):
    print(f"Module: {module.__class__.__name__}, Input shape: {input[0].shape}")

for name, module in pipe.unet.named_modules():
    if isinstance(module, torch.nn.Linear) or isinstance(module, torch.nn.Conv2d):
        module.register_forward_hook(hook_fn)

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    print(f"Step {i}, latent_model_input shape: {latent_model_input.shape}")
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds[0],
        added_cond_kwargs={
            "text_embeds": pooled_embeds,
            "time_ids": time_ids
        }
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 7. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 8. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 9. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12, 4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1, 6, i + 1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 10. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionXLPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

# --- 1. Device and RNG ---
device = "cuda"
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)

# --- 2. Load SDXL pipeline ---
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    variant="fp16"
).to(device, dtype=torch.float16)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"

# Token-level embeddings (cross-attention)
prompt_embeds, _, _, _ = pipe.encode_prompt(prompt)  # SDXL returns multiple outputs

# Pooled embeddings (for SDXL UNet 'text_time' conditioning)
text_inputs = pipe.tokenizer(prompt, return_tensors="pt").input_ids.to(device)
pooled_embeds = pipe.text_encoder(
    text_inputs,
    output_hidden_states=True
).pooler_output.to(torch.float16)

# Time IDs (required by SDXL UNet)
time_ids = torch.tensor([[1024, 1024, 0, 0, 1024, 1024]], device=device, dtype=torch.float16)  # Adjusted for 1024x1024

# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 128, 128),  # Corrected for 1024x1024
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    # Debug: Print shape to verify
    print(f"Step {i}, latent_model_input shape: {latent_model_input.shape}")
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds[0],  # Use first prompt embed
        added_cond_kwargs={
            "text_embeds": pooled_embeds,
            "time_ids": time_ids
        }
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 7. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 8. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1,2,0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 9. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12,4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1,6,i+1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 10. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")

In [ ]:
# --- 0. Setup ---
#!pip install diffusers accelerate transformers safetensors torch torchvision scikit-learn imageio -q

import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionXLPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

# --- 1. Device and RNG ---
device = "cuda"  # assume GPU is always available
generator = torch.Generator(device=device).manual_seed(1234)
print("Using device:", device)

# --- 2. Load SDXL pipeline ---
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    variant="fp16"
).to(device, dtype=torch.float16)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 3. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"

# Token-level embeddings (cross-attention)
prompt_embeds = pipe.encode_prompt(prompt)

# Pooled embeddings (for SDXL UNet 'text_time' conditioning)
text_inputs = pipe.tokenizer(prompt, return_tensors="pt").input_ids.to(device)
pooled_embeds = pipe.text_encoder(
    text_inputs,
    output_hidden_states=True
).pooler_output.to(torch.float16)

# Time IDs (required by SDXL UNet)
time_ids = torch.zeros((1, 6), device=device, dtype=torch.float16)

# --- 4. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds,
        added_cond_kwargs={
            "text_embeds": pooled_embeds,
            "time_ids": time_ids
        }
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 5. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 6. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 7. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 8. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1,2,0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 9. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12,4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1,6,i+1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 10. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")


In [ ]:
# --- 0. Setup ---
!pip install diffusers accelerate transformers safetensors torch torchvision scikit-learn imageio -q

import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionXLPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

generator = torch.Generator(device=device).manual_seed(1234)

# --- 1. Load SDXL pipeline ---
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    variant="fp16"
).to(device, dtype=torch.float16)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 2. Encode prompt ---
prompt = "a surreal city of glass towers at sunset"

# Per-token embeddings (pipeline handles tokenization internally)
prompt_embeds = pipe.encode_prompt(prompt)

# Pooled embeddings for SDXL UNet 'text_time' conditioning
text_inputs = pipe.tokenizer(prompt, return_tensors="pt").input_ids.to(device)
pooled_embeds = pipe.text_encoder(
    text_inputs,
    output_hidden_states=True
).pooler_output.to(torch.float16)

# --- 3. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds,
        added_cond_kwargs={"text_embeds": pooled_embeds}
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 4. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=z.device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 5. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 6. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 7. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=z.device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1,2,0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 8. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12,4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1,6,i+1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 9. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")


In [ ]:
# --- 0. Setup ---
!pip install diffusers accelerate transformers safetensors torch torchvision scikit-learn imageio -q

import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionXLPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# RNG
generator = torch.Generator(device=device).manual_seed(1234)

# --- 1. Load SDXL pipeline ---
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    variant="fp16"
).to(device, dtype=torch.float16)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 2. Encode prompt via pipeline ---
prompt = "a surreal city of glass towers at sunset"

# SDXL pipeline handles tokenization, text encoder projection, and pooled embeddings internally
prompt_embeds = pipe.encode_prompt(prompt)

# --- 3. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 4. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=z.device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 5. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 6. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 7. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=z.device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1,2,0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 8. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12,4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1,6,i+1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 9. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")


In [ ]:
# --- 0. Setup ---
!pip install diffusers accelerate transformers safetensors torch torchvision scikit-learn imageio

import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionXLPipeline
from sklearn.decomposition import TruncatedSVD
import imageio

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Device-aware RNG
generator = torch.Generator(device=device).manual_seed(1234)

# --- 1. Load SDXL pipeline ---
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    variant="fp16"
).to(device, dtype=torch.float16)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 2. Prompt embeddings ---
prompt = "a surreal city of glass towers at sunset"

# Tokenize
text_inputs = pipe.tokenizer(
    prompt,
    padding="max_length",
    max_length=pipe.tokenizer.model_max_length,
    truncation=True,
    return_tensors="pt"
).to(device)

# Text encoder outputs
outputs = pipe.text_encoder(
    text_inputs.input_ids,
    attention_mask=text_inputs.attention_mask,
    output_hidden_states=True,
)

prompt_embeds = outputs.last_hidden_state.to(torch.float16)       # (batch, seq_len, hidden_size)
pooled_embeds = outputs.pooler_output.to(torch.float16)           # (batch, hidden_size)

# Conditioning kwargs
add_text_embeds = pooled_embeds
add_time_ids = torch.zeros((1, 6), device=device, dtype=torch.float16)

# --- 3. Generate anchor latent ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),
    device=device,
    dtype=torch.float16,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds,
        added_cond_kwargs={
            "text_embeds": add_text_embeds,
            "time_ids": add_time_ids,
        }
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 4. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=z.device, dtype=torch.float16)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 5. Decode probes ---
with torch.no_grad():
    decoded = vae.decode((z_probes / vae.config.scaling_factor).half()).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 6. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 7. Animate along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=z.device, dtype=torch.float16)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode((z / vae.config.scaling_factor).half()).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append((img[0].permute(1,2,0).cpu().numpy() * 255).astype(np.uint8))
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 8. Display first frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12,4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1,6,i+1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "BIG singular mode")
show_frames(frames_small, "SMALL singular mode")

# --- 9. Export GIFs ---
imageio.mimsave("animation_big.gif", frames_big, duration=0.2)
imageio.mimsave("animation_small.gif", frames_small, duration=0.2)

print("GIFs saved: animation_big.gif, animation_small.gif")


In [ ]:
# --- 0. Setup ---

import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionXLPipeline
from sklearn.decomposition import TruncatedSVD

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Device-aware RNG
generator = torch.Generator(device=device).manual_seed(1234)

# --- 1. Load SDXL pipeline ---
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    variant="fp16"
).to(device, dtype=torch.float16)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 2. Prompt embeddings ---
prompt = "a surreal city of glass towers at sunset"

# Tokenize
text_inputs = pipe.tokenizer(
    prompt,
    padding="max_length",
    max_length=pipe.tokenizer.model_max_length,
    truncation=True,
    return_tensors="pt"
).to(device)

# Text encoder outputs
outputs = pipe.text_encoder(
    text_inputs.input_ids,
    attention_mask=text_inputs.attention_mask,
    output_hidden_states=True,
)

prompt_embeds = outputs.last_hidden_state       # (batch, seq_len, hidden_size)
pooled_embeds = outputs.pooler_output          # (batch, hidden_size)

# Conditioning kwargs for UNet
add_text_embeds = pooled_embeds
add_time_ids = torch.zeros((1, 6), device=device, dtype=prompt_embeds.dtype)

# --- 3. Generate anchor latent from prompt ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),
    device=device,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds,
        added_cond_kwargs={
            "text_embeds": add_text_embeds,
            "time_ids": add_time_ids,
        }
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 4. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=z.device)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 5. Decode probes ---
with torch.no_grad():
    decoded = vae.decode(z_probes / vae.config.scaling_factor).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 6. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 7. Animate by stepping along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    # heuristic: map flat image-space singular vector back into latent-shape
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=z.device)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode(z / vae.config.scaling_factor).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append(img[0].permute(1,2,0).cpu().numpy())
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 8. Display frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12,4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1,6,i+1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "Animation along BIG singular mode")
show_frames(frames_small, "Animation along SMALL singular mode")


In [ ]:
# --- 0. Setup ---
#!pip install diffusers accelerate transformers safetensors torch torchvision scikit-learn -q

import torch
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionXLPipeline
from sklearn.decomposition import TruncatedSVD

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Device-aware RNG
generator = torch.Generator(device=device).manual_seed(1234)

# --- 1. Load SDXL pipeline ---
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    variant="fp16"
).to(device, dtype=torch.float16)

pipe.enable_vae_tiling()
vae = pipe.vae

# --- 2. Prompt embeddings ---
prompt = "a surreal city of glass towers at sunset"

# SDXL encode_prompt returns a single tensor
prompt_embeds = pipe.encode_prompt(prompt)

# pooled embeddings come from the *text encoder’s pooled output*
# pipe.encode_prompt does not expose it directly
# instead, we use the tokenizer + text_encoder
text_inputs = pipe.tokenizer(
    prompt,
    padding="max_length",
    max_length=pipe.tokenizer.model_max_length,
    truncation=True,
    return_tensors="pt"
).to(device)

prompt_embeds = pipe.text_encoder(
    text_inputs.input_ids,
    attention_mask=text_inputs.attention_mask,
)[0]  # last hidden states

pooled_embeds = pipe.text_encoder(
    text_inputs.input_ids,
    attention_mask=text_inputs.attention_mask,
    output_hidden_states=True,
)[1][-1][:, 0]  # pooled CLS-like embedding

# --- 3. Conditioning kwargs ---
add_text_embeds = pooled_embeds
add_time_ids = torch.zeros((1, 6), device=device, dtype=prompt_embeds.dtype)

# --- 3b. Generate anchor latent from prompt ---
num_inference_steps = 30
latents = torch.randn(
    (1, pipe.unet.config.in_channels, 64, 64),
    device=device,
    generator=generator
)

pipe.scheduler.set_timesteps(num_inference_steps, device=device)
z_anchor = None

for i, t in enumerate(pipe.scheduler.timesteps):
    latent_model_input = pipe.scheduler.scale_model_input(latents, t)
    noise_pred = pipe.unet(
        latent_model_input,
        t,
        encoder_hidden_states=prompt_embeds,
        added_cond_kwargs={
            "text_embeds": add_text_embeds,
            "time_ids": add_time_ids,
        }
    ).sample
    step_out = pipe.scheduler.step(noise_pred, t, latents)
    latents = step_out.prev_sample if hasattr(step_out, "prev_sample") else step_out

    if i == 20:  # mid-step anchor
        z_anchor = latents.clone().detach()
        break

if z_anchor is None:
    z_anchor = latents.clone().detach()

# --- 4. Proximal probes ---
def make_probes(z, n_probes=32, eps=0.05):
    probes = []
    for _ in range(n_probes):
        noise = torch.randn_like(z, device=z.device)
        noise = noise / (noise.norm() + 1e-12) * (eps * (z.norm() + 1e-12))
        probes.append(z + noise)
    return torch.cat(probes, dim=0)

z_probes = make_probes(z_anchor, n_probes=32, eps=0.05)

# --- 5. Decode probes ---
with torch.no_grad():
    decoded = vae.decode(z_probes / vae.config.scaling_factor).sample

imgs = (decoded.clamp(-1, 1) + 1) / 2
X = imgs.cpu().flatten(start_dim=1).float().numpy()

# --- 6. SVD resonance analysis ---
svd = TruncatedSVD(n_components=10)
svd.fit(X)
singular_vectors = svd.components_
print("Explained variance ratios:", svd.explained_variance_ratio_)

# --- 7. Animate by stepping along singular directions ---
def traverse_singular(z_start, direction_flat, n_frames=12, step_size=0.05):
    # heuristic: map flat image-space singular vector back into latent-shape
    z = z_start.clone().detach()
    latent_dir = torch.randn_like(z, device=z.device)
    latent_dir = latent_dir / (latent_dir.norm() + 1e-12) * np.linalg.norm(direction_flat)
    frames = []
    for i in range(n_frames):
        z = z + step_size * latent_dir
        with torch.no_grad():
            out = vae.decode(z / vae.config.scaling_factor).sample
        img = (out.clamp(-1, 1) + 1) / 2
        frames.append(img[0].permute(1,2,0).cpu().numpy())
    return frames

big_mode = singular_vectors[0]
small_mode = singular_vectors[-1]

frames_big = traverse_singular(z_anchor, big_mode, n_frames=12, step_size=0.05)
frames_small = traverse_singular(z_anchor, small_mode, n_frames=12, step_size=0.05)

# --- 8. Display frames ---
def show_frames(frames, title):
    plt.figure(figsize=(12,4))
    for i, f in enumerate(frames[:6]):
        plt.subplot(1,6,i+1)
        plt.imshow(f)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

show_frames(frames_big, "Animation along BIG singular mode")
show_frames(frames_small, "Animation along SMALL singular mode")


# Task
Explain the error in the selected code. If possible, fix the error and incorporate the changes into the existing code. Otherwise, try to diagnose the error. Draw inspiration from the colab notebook at "https://github.com/TheOneTrueGuy/DigitalDayDream/blob/main/1simple_SDXL_colab.ipynb" while still keeping the Aproximal Resonance Mapping features.

## Examine the provided notebook

### Subtask:
Analyze the Colab notebook at `https://github.com/TheOneTrueGuy/DigitalDayDream/blob/main/1simple_SDXL_colab.ipynb` to identify how it sets up the SDXL pipeline, handles memory, and performs its image generation. Pay close attention to pipeline loading, data types, memory optimization flags, and the overall structure of the diffusion loop.
